### Задание 1 - 10 баллов

- Загрузить набор данных по выбору с помощью библиотеки Corus - **1 балл**
- Провести релевантную предобработку выбранного датасета: - **6 баллов**
  - Нормализация
  - Токенизация
  - Удаление стоп-слов
  - Лемматизация/стемминг

- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **2 балла**

- Соблюден code style на уровне pep8 и [On writing clean Jupyter notebooks](https://ploomber.io/blog/clean-nbs/) - **1 балл**

Инструменты для решения задач предобработки – NLTK, Gensim, Natasha, pymystem, pymorphy2…

Для сдачи ДЗ - приложите ссылку на PR (Pull Request) из ветки hw_1 в ветку main в вашем приватном репозитории на github.com

In [1]:
import re

import nltk
import requests
from bs4 import BeautifulSoup
from corus import load_rudrec
from nltk.corpus import stopwords

from hw1_utils import tokenize_and_filter

In [ ]:
# скачивание всего необходимого
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
nltk.download('punkt')
nltk.download("stopwords")

In [3]:
# загрузка датасета
path = './rudrec_annotated.json'
corpus = load_rudrec(path)
texts = [record.text for record in corpus]

In [4]:
# нормирование регуляркой, оставляем только кирилицу и несколько символов пунктуации
# в данном датасете для анализа отзывов на лекарства нам вряд ли нужны цифры

allowed_symbols = re.compile(u'[0-9]|[^\u0400-\u04FF \-]')
ru_stopwords = stopwords.words("russian")

normed_texts = [allowed_symbols.sub(' ', text).lower()
                for text
                in texts]
# токенизация, лемматизация и постпроцессинг
tokenised = [tokenize_and_filter(text, ru_stopwords)
             for text
             in normed_texts]

Спарсим из интереса названия лекарств с сайта аптеки и посмотрим, на что чаще всего отзывы

In [6]:
url = "https://apteki.medsi.ru/products/"

response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "lxml")

all_drugs_items = soup.find_all('li', 'drug-list__item')
all_drugs_names = [item.find('a', 'drug-list__link').text.lower()
                   for item
                   in all_drugs_items]

print(f'Всего наименований: {len(all_drugs_names)}')

Всего наименований: 6770


In [7]:
# посчитаем сколько раз в датасете попадалось то или иное название
drug_stats = dict.fromkeys(all_drugs_names, 0)

for token_list in tokenised:
    for token in token_list:
        if token in all_drugs_names:
            drug_stats[token] += 1

In [9]:
# отфильтруем нули
non_zero_stats = [(name, count)
                  for name, count
                  in drug_stats.items()
                  if count != 0]

sorted_by_number = sorted(non_zero_stats, key=lambda tup: tup[1], reverse=True)
print(f'Всего упоминалось: {len(sorted_by_number)}')

Всего упоминалось: 115


In [10]:
sorted_by_number[:10]

[('виферон', 53),
 ('анаферон', 47),
 ('циклоферон', 41),
 ('глицин', 26),
 ('кагоцел', 25),
 ('ацикловир', 22),
 ('валериана', 22),
 ('ликопид', 22),
 ('гриппферон', 20),
 ('амизон', 17)]